In [11]:
from tree_manager import Tree, TreeNode, TreeEdge, init_db, SessionLocal
init_db()
session = SessionLocal()

In [12]:
tree = Tree.get(session, id=1)
new_tag = tree.create_tag(session, "release-v1.0", description="Initial stable release")

In [13]:
modified_tree = tree.create_new_tree_version_from_tag(session, "release-v1.0")
new_node = modified_tree.add_node(session, data={"setting": "new_value"})
modified_tree.create_tag(session, "release-v1.1", description="Added new setting")

In [14]:

# Adding an edge between nodes
modified_tree.add_edge(session, incoming_node_id=1, outgoing_node_id=2, data={"weight": 0.5})

# Retrieving historical configuration
old_config = tree.get_by_tag(session, "release-v1.0")

In [15]:
main_tree = tree.get_by_tag(session, "release-v1.0")
feature_branch = main_tree.create_new_tree_version_from_tag(session, "release-v1.0")

In [16]:
node1 = feature_branch.add_node(session, data={"feature_flag": True})
node2 = feature_branch.add_node(session, data={"config": "new_setting"})
feature_branch.add_edge(session, incoming_node_id=node1.id, outgoing_node_id=node2.id, data={"relation": "depends_on"})
feature_branch.create_tag(session, "feature-x-v1", description="Feature X implementation")

In [17]:
stable_tree = Tree.get(session, id=1)
stable_tag = stable_tree.create_tag(session,"stable-v1")
new_node = stable_tree.add_node(session, data={"experimental": True})
stable_tree.add_edge(session, incoming_node_id=1, outgoing_node_id=new_node.id, data={"type": "experimental"})


In [18]:
rollback_tree = stable_tree.restore_from_tag(session, "stable-v1")


In [19]:
rollback_tree.nodes

In [20]:
stable_tree.nodes

In [23]:
historical_tree = tree.get_by_tag(session, "release-v1.0")


In [26]:
root_node = historical_tree.get_root_nodes(session)


In [29]:
node = historical_tree.get_node(session, node_id=1)
children = historical_tree.get_child_nodes(session, node_id=1)
parents = historical_tree.get_parent_nodes(session, node_id=1)

In [30]:
edges = historical_tree.get_node_edges(session, node_id=1)
for edge in edges:
    print(f"Edge {edge.id} metadata: {edge.data}")
    print(f"Connected to node: {edge.outgoing_node_id}")

Edge 1 metadata: {'relation': 'child'}
Connected to node: 2
Edge 2 metadata: {'relation': 'child'}
Connected to node: 3
Edge 5 metadata: {'relation': 'child'}
Connected to node: 6
Edge 17 metadata: {'weight': 0.5}
Connected to node: 2
Edge 25 metadata: {'type': 'experimental'}
Connected to node: 29


In [35]:
level_2_nodes = historical_tree.get_nodes_at_depth(session,2)


In [37]:
path = historical_tree.find_path(session, start_node_id=1, end_node_id=5)
for node, edge in path:
    print(f"Node: {node.data}")
    print(f"Connected by edge: {edge.data if edge else 'None'}")

Node: {'name': 'Root Node', 'setting': 'default'}
Connected by edge: {'relation': 'child'}
Node: {'name': 'Child Node 1', 'setting': 'custom'}
Connected by edge: {'relation': 'grandchild'}
Node: {'name': 'Grandchild Node 2', 'setting': 'experimental'}
Connected by edge: None
